In [1]:
!pip install pandas scikit-learn nltk

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
import re
import os

In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

In [ ]:
#replace with your own file path
file_path = r"C:\Users\user\Downloads\reddit_sentiment(farah)\reddit_sentiment\data\raw_data\reddit_malaysian_20250626_014418.jsonl"
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame(data)
if 'content' not in df.columns:
    raise ValueError("The JSONL file does not contain a 'content' field for comments.")
df.head()

,id,title,content,score,num_comments,created_utc,created_date,author,subreddit,url,timestamp,fetch_method,content_type,parent_post_id
0,1lka6bd,Is sexual health/risks not taken seriously her...,Let’s discuss!\ni’m quite a young girl and i n...,5,9,1.750869e+09,2025-06-25T16:23:52+00:00,hoodiesnake,malaysians,https://www.reddit.com/r/malaysians/comments/1...,2025-06-25T17:43:45.454692+00:00,reddit_api,post,NaN
1,mzq3w4z,Comment on: Is sexual health/risks not taken s...,You should get tested honestly. If you're worr...,2,0,1.750870e+09,2025-06-25T16:41:13+00:00,Upbeat-Vibe-Chill,malaysians,https://reddit.com/r/malaysians/comments/1lka6...,2025-06-25T17:43:46.141779+00:00,reddit_api,comment,1lka6bd
2,mzq411u,Comment on: Is sexual health/risks not taken s...,RIP your DMs...,1,0,1.750870e+09,2025-06-25T16:41:52+00:00,-OddLion-,malaysians,https://reddit.com/r/malaysians/comments/1lka6...,2025-06-25T17:43:46.145784+00:00,reddit_api,comment,1lka6bd
3,mzqb4gy,Comment on: Is sexual health/risks not taken s...,"If you're not comfortable, don't agree to sex ...",1,0,1.750872e+09,2025-06-25T17:14:15+00:00,ThosaiWithCheese,malaysians,https://reddit.com/r/malaysians/comments/1lka6...,2025-06-25T17:43:46.149303+00:00,reddit_api,comment,1lka6bd
4,mzqf3f0,Comment on: Is sexual health/risks not taken s...,Saw your post history,1,0,1.750873e+09,2025-06-25T17:31:58+00:00,Lunartic2102,malaysians,https://reddit.com/r/malaysians/comments/1lka6...,2025-06-25T17:43:46.155175+00:00,reddit_api,comment,1lka6bd


In [22]:
# Drop rows where content is missing or empty
df = df[df['content'].notna() & (df['content'].str.strip() != '')]
df['clean_comment'] = df['content'].apply(clean_text)
df[['content', 'clean_comment']].head()

,content,clean_comment
0,Let’s discuss!\ni’m quite a young girl and i n...,lets discuss im quite young girl never proper ...
1,You should get tested honestly. If you're worr...,get tested honestly youre worried get tested
2,RIP your DMs...,rip dms
3,"If you're not comfortable, don't agree to sex ...",youre comfortable dont agree sex till get prop...
4,Saw your post history,saw post history


In [23]:
positive_words = ['good', 'great', 'excellent', 'happy', 'love', 'best', 'amazing', 'nice', 'wonderful', 'awesome']
negative_words = ['bad', 'terrible', 'worst', 'sad', 'hate', 'awful', 'horrible', 'poor', 'angry', 'disappointed']

def label_sentiment(text):
    positive_count = sum(1 for word in positive_words if word in text)
    negative_count = sum(1 for word in negative_words if word in text)
    if positive_count > negative_count:
        return 0
    elif negative_count > positive_count:
        return 1
    else:
        return 3

df['sentiment'] = df['clean_comment'].apply(label_sentiment)
df[['content', 'clean_comment', 'sentiment']].head()

,content,clean_comment,sentiment
0,Let’s discuss!\ni’m quite a young girl and i n...,lets discuss im quite young girl never proper ...,3
1,You should get tested honestly. If you're worr...,get tested honestly youre worried get tested,3
2,RIP your DMs...,rip dms,3
3,"If you're not comfortable, don't agree to sex ...",youre comfortable dont agree sex till get prop...,3
4,Saw your post history,saw post history,3


In [24]:
output_path = 'labeled_comments(malaysian2).csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Sentiment labeling complete. Labeled data saved to {output_path}")

Sentiment labeling complete. Labeled data saved to labeled_comments(malaysian2).csv
